In [10]:
import pandas as pd
from collections import defaultdict, Counter
from mlxtend.frequent_patterns import fpgrowth
from mlxtend.preprocessing import TransactionEncoder

In [11]:
# Lista de transações
transactions = [
    ['I1', 'I2', 'I5'],
    ['I2', 'I4'],
    ['I2', 'I3'],
    ['I1', 'I2', 'I4'],
    ['I1', 'I3'],
    ['I2', 'I3'],
    ['I1', 'I3'],
    ['I1', 'I2', 'I3', 'I5'],
    ['I1', 'I2', 'I3']
]

# Implementação explícita

In [12]:
supMin = 2
n_trans = len(transactions)

# Etapa 1: Contar e filtrar itens
item_counts = Counter()
for t in transactions:
    item_counts.update(t)

item_counts = {item: count for item, count in item_counts.items() if count >= supMin}

# Etapa 2: Reordenar transações
def sort_items(t):
    return [item for item in sorted(t, key=lambda x: (-item_counts.get(x, 0), x)) if item in item_counts]

transactions = [sort_items(t) for t in transactions if sort_items(t)]

# Estrutura de nó da FP-Tree
class FPNode:
    def __init__(self, item, parent):
        self.item = item
        self.count = 0
        self.parent = parent
        self.children = {}
        self.link = None

    def increment(self, count=1):
        self.count += count

# Inserção de transações na FP-Tree
def insert_tree(t, node, header_table, count=1):
    if not t:
        return
    first, rest = t[0], t[1:]
    if first in node.children:
        node.children[first].increment(count)
    else:
        child = FPNode(first, node)
        child.increment(count)
        node.children[first] = child
        # Atualizar header_table
        if header_table[first][1] is None:
            header_table[first][1] = child
        else:
            current = header_table[first][1]
            while current.link:
                current = current.link
            current.link = child
    insert_tree(rest, node.children[first], header_table, count)

# Construção da árvore principal
def build_fp_tree(transactions, item_counts):
    header_table = {item: [0, None] for item in item_counts}
    root = FPNode(None, None)
    for t in transactions:
        insert_tree(t, root, header_table)
        for item in t:
            header_table[item][0] += 1
    return root, header_table

# Caminho até a raiz
def ascend_tree(node):
    path = []
    while node and node.parent and node.parent.item is not None:
        node = node.parent
        path.append(node.item)
    return path

# Caminhos prefixados
def find_prefix_paths(base_item, header_table):
    node = header_table[base_item][1]
    cond_pats = []
    while node:
        path = ascend_tree(node)
        if path:
            cond_pats.append((path[::-1], node.count))
        node = node.link
    return cond_pats

# Inserção em árvore condicional
def insert_conditional_tree(t, node, header_table, count=1):
    if not t:
        return
    first, rest = t[0], t[1:]
    if first in node.children:
        node.children[first].increment(count)
    else:
        child = FPNode(first, node)
        child.count = count
        node.children[first] = child
        if header_table[first][1] is None:
            header_table[first][1] = child
        else:
            current = header_table[first][1]
            while current.link:
                current = current.link
            current.link = child
    insert_conditional_tree(rest, node.children[first], header_table, count)

# Mineração da FP-Tree
def mine_tree(header_table, prefix, freq_itemsets):
    sorted_items = sorted(header_table.items(), key=lambda x: x[1][0])  # ordem crescente de frequência

    for item, (initial_support, node) in sorted_items:
        new_prefix = prefix + [item]

        # Suporte total do novo itemset = soma dos nós linkados ao item
        total_support = 0
        current_node = node
        while current_node:
            total_support += current_node.count
            current_node = current_node.link

        if total_support >= supMin:
            freq_itemsets.append((set(new_prefix), total_support))

        # Gerar caminhos prefixados condicionais
        cond_paths = find_prefix_paths(item, header_table)

        # Contagem de itens nos caminhos condicionais
        cond_counts = Counter()
        for path, cnt in cond_paths:
            cond_counts.update({i: cnt for i in path})

        # Filtrar itens com suporte ≥ supMin
        cond_counts = {i: c for i, c in cond_counts.items() if c >= supMin}
        if not cond_counts:
            continue

        # Construir árvore condicional
        cond_header = {i: [0, None] for i in cond_counts}
        cond_root = FPNode(None, None)
        for path, cnt in cond_paths:
            filtered_path = [i for i in path if i in cond_counts]
            filtered_path.sort(key=lambda x: (-cond_counts[x], x))
            insert_conditional_tree(filtered_path, cond_root, cond_header, cnt)

        # Recursão
        mine_tree(cond_header, new_prefix, freq_itemsets)

# Execução principal
root, header_table = build_fp_tree(transactions, item_counts)
freq_itemsets = []
mine_tree(header_table, [], freq_itemsets)

# Resultados
df_fp = pd.DataFrame([
    {'itemsets': itemset, 'support': support / n_trans, 'length': len(itemset)}
    for itemset, support in freq_itemsets
])
df_fp = df_fp.sort_values(by=['length', 'support'], ascending=[True, False]).reset_index(drop=True)

df_fp

,itemsets,support,length
0,{I2},0.777778,1
1,{I1},0.666667,1
2,{I3},0.666667,1
3,{I5},0.222222,1
4,{I4},0.222222,1
5,"{I1, I2}",0.444444,2
6,"{I2, I3}",0.444444,2
7,"{I1, I3}",0.444444,2
8,"{I2, I5}",0.222222,2
9,"{I1, I5}",0.222222,2


# Usando o pacote mlxtend

In [13]:
# Transformar para formato one-hot
te = TransactionEncoder()
te_ary = te.fit(transactions).transform(transactions)
df = pd.DataFrame(te_ary, columns=te.columns_)

# Executar FP-Growth com suporte mínimo = 2/9 (~0.22)
frequent_itemsets = fpgrowth(df, min_support=2/9, use_colnames=True)

# Adicionar coluna 'length' como no Apriori
frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))
frequent_itemsets = frequent_itemsets.sort_values(by=['length', 'support'], ascending=[True, False])

frequent_itemsets

,support,itemsets,length
0,0.777778,(I2),1
1,0.666667,(I1),1
4,0.666667,(I3),1
2,0.222222,(I5),1
3,0.222222,(I4),1
5,0.444444,"(I1, I2)",2
10,0.444444,"(I2, I3)",2
11,0.444444,"(I1, I3)",2
6,0.222222,"(I1, I5)",2
7,0.222222,"(I2, I5)",2
